# SQL 활용 데이터 프레임 생성

> Python과 Oracle 연동

## #01. 패키지

`oracledb`와 `sqlalchemy` 패키지 설치

```shell
pip install cx_oracle
```

In [ ]:
import cx_Oracle as cx
from pandas import DataFrame, read_sql_table
from sqlalchemy import create_engine

## #02. cx_Oracle 사용

### 1. DB 접속

In [ ]:
dbcon = cx.connect(
    'hr', # 사용자 이름
    'hr', # 비밀번호
    'localhost:1521/xe' # DB 서버 주소
)
dbcon

### 2. 데이터 조회

#### 기본 사용 방법

테이블의 각 record를 튜플로 표현하는 리스트 객체를 얻을 수 있다.

##### 데이터 조회를 위한 커서 객체 생성

In [ ]:
cursor = dbcon.cursor()

##### 데이터 조회를 위한 SQL문 처리

In [ ]:
sql = 'SELECT * FROM department'
cursor.execute(sql)
result = cursor.fetchall()
result

##### 딕셔너리 형태로 데이터 조회

cx_oracle은 별도의 딕셔너리 형태를 제공하지 않기 때문에 cursor 객체의 rowfactory 프로퍼티를 직접 재정의 해야 한다.

cursor.rowfactory = lambda *args: dict(zip([d[0] for d in cursor.description], args))

> 코드출처 : oracle.com

In [ ]:
sql = 'SELECT * FROM department ORDER BY deptno ASC'    # 소문자를 넣어도 필드 이름은 대문자로 추출
cursor.execute(sql)

cursor.rowfactory = lambda *args: dict(zip([d[0] for d in cursor.description], args))
result = cursor.fetchall()
result

##### 조회 결과를 DF로 변환

In [ ]:
df = DataFrame(result)
df

##### DF에 대한 인덱스 설정

In [ ]:
df.set_index('DEPTNO', inplace = True)
df

### #3. 입력, 수정, 삭제

`INSERT`, `UPDATE`, `DELETE` 문의 수행 방식은 동일

>여기서는 데이터 조회 과정에서 생성한 cursor 객체를 재사용

#### 데이터 입력

In [ ]:
sql = 'SELECT * FROM department.nextval from dual'    # 소문자를 넣어도 필드 이름은 대문자로 추출
cursor.execute(sql)

result = cursor.fetchall()
print(result)

seq = result[0][0]
print("새로운 시퀀스 번호 : %d" % seq)

In [ ]:
sql = 'INSERT INTO department (deptno, dname, loc) VALUES (:1, :2, :3)'
print(sql)

#                      :1    :2      :3
cursor.execute(sql, [seq, '개발부', '서울'])
print("%s개의 행 저장"%cursor.rowcount)

# 처리 결과 실제로 반영
dbcon.commit()

# 되돌리기
# --> 이미 commit한 내역은 적용 불가
# dbcon.rollback()

#### 데이터 수정

In [ ]:
sql = 'UPDATE department SET dname =: 1, loc=:2 where deptno=:3'
cursor.execute(sql, ['영업부', '부산', seq])
print("%s개의 행 갱신"%cursor.rowcount)
dbcon.commit()  # commit 해야 DB에 반영


#### 데이터 삭제

In [ ]:
sql = 'DELETE FROM department WHERE deptno > 202'
cursor.execute(sql)
print("%s개의 행 삭제"%cursor.rowcount)
dbcon.commit()  # commit 해야 DB에 반영


#### DB 접속 해제

In [ ]:
cursor.close()
dbcon.close()

## #03 SQLAIchemy 사용

### #1. 데이터베이스 접속

접속 문자열 생성

`oracle+cx_oracle://계정이름:비밀번호@접속주소/SID`

In [ ]:
conStr = "oracle+cx_oracle://hr:hr@localhost:1521/xe"

#### DB 접속

In [ ]:
engine = create_engine(conStr)
conn = engine.connect()

### #2. 데이터 조회

#### 특정 테이블의 모든 데이터 조회

In [ ]:
df = read_sql_table('department', con=conn)
df

#### 인덱스를 지정한 조회

> read_sql_table 함수를 사용할 경우 where 절 사용 불가

In [ ]:
df = read_sql_table('department', index_col='deptno', con=conn)
df

#### 특정 컬럼 조회

In [ ]:
df = read_sql_table('department', index_col='deptno', columns=['dname'], con=conn)
df

### #3. 데이터 내보내기

- name='테이블명' 이름으로 기존 테이블이 있으면 해당 테이블의 컬럼명에 맞게 데이터를 넣을 수 있음
- if_exists='append' 옵션이 있으면, 기존 테이블에 데이터를 추가로 넣음
- if_exists='fail' 옵션이 있으면, 기존 테이블이 있을 경우, 아무일도 하지 않음
- if_exists='replace' 옵션이 있으면, 기존 테이블이 있을 경우, 기존 테이블을 삭제하고, 다시 테이블을 만들어서, 새로 데이터를 넣음

이미 만들어진 테이블이 없으면, name='테이블명' 이름으로 테이블을 자동으로 만들고, 데이터를 넣을 수 있음

테이블이 자동으로 만들어지므로, 테이블 구조가 최적화되지 않아 자동으로 테이블 만드는 것은 추천하지 않음

In [ ]:
df.to_sql('new_table', con=conn, if_exists='append', index=False)
conn.commit()

### #4. DB 접속 해제

DB 관련 작업 종료 시 반드시 접속 객체 반납

In [ ]:
conn.close()